<a href="https://colab.research.google.com/github/harish-git63/Demo1/blob/main/Resturant_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
from tkinter import *
from tkinter import ttk
from tkinter import messagebox

# ---------------------- DATABASE SETUP ----------------------
def connect_db():
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS orders (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            customer_name TEXT NOT NULL,
            item TEXT NOT NULL,
            quantity INTEGER NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

# ---------------------- FUNCTIONS ----------------------
def add_order():
    name = entry_name.get()
    item = entry_item.get()
    quantity = entry_quantity.get()

    if not name or not item or not quantity:
        messagebox.showwarning("Input Error", "All fields are required!")
        return

    try:
        quantity = int(quantity)
    except ValueError:
        messagebox.showwarning("Input Error", "Quantity must be a number!")
        return

    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO orders (customer_name, item, quantity) VALUES (?, ?, ?)",
                   (name, item, quantity))
    conn.commit()
    conn.close()

    view_orders()
    clear_fields()

def view_orders():
    for row in tree.get_children():
        tree.delete(row)

    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM orders")
    rows = cursor.fetchall()
    for row in rows:
        tree.insert("", END, values=row)
    conn.close()

def delete_order():
    selected = tree.selection()
    if not selected:
        messagebox.showwarning("Select an order", "Please select an order to delete.")
        return

    order_id = tree.item(selected[0])['values'][0]

    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM orders WHERE id=?", (order_id,))
    conn.commit()
    conn.close()

    view_orders()

def clear_fields():
    entry_name.delete(0, END)
    entry_item.set(item_options[0])
    entry_quantity.delete(0, END)

# ---------------------- UI SETUP ----------------------
connect_db()
root = Tk()
root.title("RESTAURANT ORDER SYSTEM")
root.geometry("700x600")
root.configure(bg='#FFFACD')  # Light Yellow

# Title
title_frame = Frame(root, bg='#1E90FF', bd=3, relief="solid")  # Dodger Blue
title_frame.pack(fill=X, pady=10, padx=10)
Label(
    title_frame,
    text="RESTAURANT ORDER SYSTEM",
    font=("Arial", 20, "bold"),
    fg="white",
    bg='#1E90FF'
).pack(pady=10)

# Center Form Frame
form_frame = Frame(root, bg='#FFFACD')  # Light Yellow
form_frame.pack(pady=20)

Label(form_frame, text="Customer Name", bg='#FFFACD', font=("Arial", 12)).pack(pady=5)
entry_name = Entry(form_frame, width=30, font=("Arial", 12))
entry_name.pack()

Label(form_frame, text="Item", bg='#FFFACD', font=("Arial", 12)).pack(pady=5)
item_options = ["Dosa", "Idli Vada", "ALoo Parata", "Masala Dosa", "Poori", "Medu Vada"]
entry_item = ttk.Combobox(form_frame, values=item_options, state="readonly", width=28, font=("Arial", 12))
entry_item.pack()
entry_item.set(item_options[0])

Label(form_frame, text="Quantity", bg='#FFFACD', font=("Arial", 12)).pack(pady=5)
entry_quantity = Entry(form_frame, width=30, font=("Arial", 12))
entry_quantity.pack()

# Button Frame (Centered)
btn_frame = Frame(root, bg='#FFFACD')
btn_frame.pack(pady=20)

btn_style = {
    "font": ("Arial", 11, "bold"),
    "bg": "#FF6347",  # Tomato red
    "fg": "white",
    "activebackground": "#FFD700",  # Gold
    "activeforeground": "black"
}

Button(btn_frame, text="Add Order", width=15, command=add_order, **btn_style).grid(row=0, column=0, padx=10)
Button(btn_frame, text="View Orders", width=15, command=view_orders, **btn_style).grid(row=0, column=1, padx=10)
Button(btn_frame, text="Delete Order", width=15, command=delete_order, **btn_style).grid(row=0, column=2, padx=10)

# Treeview Style
style = ttk.Style()
style.theme_use("default")
style.configure("Treeview",
                background="#f0f0f0",
                foreground="black",
                rowheight=25,
                fieldbackground="#f0f0f0",
                font=('Arial', 11))
style.map('Treeview', background=[('selected', '#ADD8E6')])  # Light Blue on select

tree_frame = Frame(root)
tree_frame.pack(pady=10, padx=20, fill=BOTH, expand=True)

tree = ttk.Treeview(tree_frame, columns=("ID", "Name", "Item", "Quantity"), show='headings')
tree.heading("ID", text="ID")
tree.heading("Name", text="Customer Name")
tree.heading("Item", text="Item")
tree.heading("Quantity", text="Quantity")

tree.column("ID", anchor=CENTER, width=50)
tree.column("Name", anchor=CENTER, width=150)
tree.column("Item", anchor=CENTER, width=150)
tree.column("Quantity", anchor=CENTER, width=100)

tree.pack(fill=BOTH, expand=True)

# Start
view_orders()
root.mainloop()

TclError: no display name and no $DISPLAY environment variable